In [1]:
import ROOT
import requests
import findspark
findspark.init('/usr/local/spark')
from pyspark import SparkConf, SparkContext

Welcome to JupyROOT 6.24/02


In [2]:
# check some of the env variables
!env | grep -i spark

SPARK_HOME=/usr/local/spark
PYSPARK_PYTHON=/root/miniconda3/envs/test/bin/python
PATH=/root/miniconda3/envs/test/bin:/root/miniconda3/condabin:/usr/lib64/qt-3.3/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/usr/local/spark/bin:/root/bin:/usr/local/spark/bin
stopspark=/usr/local/spark/sbin/stop-all.sh
startspark=/usr/local/spark/sbin/start-all.sh


In [3]:
buck = 'https://cloud-areapd.pd.infn.it:5210/swift/v1/AUTH_d2e941ce4b324467b6b3d467a923a9bc/LCPmodB-Y3_CMS_FCNC/'

local = '/data/FCNC/'

tofile = 'SingleMuon_Run2016B_ver1-Nano1June2019_ver1-v1/4F32E8E2-3E72-E84D-A12B-CA4CB942D65C_Skim.root'

#### Trials with ROOT documentation

In [4]:
#import boto3

#s3 = boto3.resource('s3')
#bucket = s3.Bucket(buck)
#cl = boto3.client('s3')
#response = cl.list_objects(Bucket=buck)

#bucket.objects()

In [4]:
# Create a SparkContext object with the right configuration for your Spark cluster
conf = SparkConf().setAppName('appName').setMaster('spark://10.67.22.59:7077')
sc = SparkContext(conf=conf)

2021-07-06 12:37:02 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
file_list_raw = requests.get(buck, verify=False)
file_list = file_list_raw.text.split('\n')

/root/miniconda3/envs/test/lib/python3.6/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloud-areapd.pd.infn.it'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [6]:
for i in range(len(file_list)):
    file_list[i] = 's3' + buck + file_list[i]
    
chain = ROOT.TChain('Events')
for file in file_list[:2]:
    chain.AddFile(file)

In [7]:
# Point RDataFrame calls to the Spark specific RDataFrame
RDataFrame = ROOT.RDF.Experimental.Distributed.Spark.RDataFrame
 
# The Spark RDataFrame constructor accepts an optional "sparkcontext" parameter
# and it will distribute the application to the connected cluster
df = RDataFrame(chain, sparkcontext = sc)

In [8]:
def count(df):
    import findspark
    findspark.init('/usr/local/spark')
    
    return df.Count().GetValue()

In [12]:
count(df)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 4 times, most recent failure: Lost task 0.3 in stage 3.0 (TID 31, 10.67.22.232, executor 1): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$3.applyOrElse(PythonRunner.scala:486)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$3.applyOrElse(PythonRunner.scala:475)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:36)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:593)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:571)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:406)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIt

In [12]:
df.Count().GetValue()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 2.0 failed 4 times, most recent failure: Lost task 1.3 in stage 2.0 (TID 31, 10.67.22.233, executor 2): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$3.applyOrElse(PythonRunner.scala:486)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$3.applyOrElse(PythonRunner.scala:475)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:36)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:593)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:571)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:406)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIt

In [10]:
# why doesnt it work?
sums = df.Filter("nMuon > 2").Sum("nElectron")
h = df.Histo1D("nMuon")
 
print(sums)
h.Draw()

#### Trials with DIANA-HEP

In [15]:
from pyspark.sql import SparkSession
# from pyspark.sql import SQLContext

In [ ]:
spark = SparkSession.builder \
        .master('spark://10.67.22.59:7077') \
        .appName('3rd') \
        .config('spark.jars.packages',
                'org.diana-hep:spark-root_2.11:0.1.13,org.diana-hep:histogrammar-sparksql_2.11:1.0.4,org.apache.hadoop:hadoop-aws:2.7.0',)\
        .config('spark.cores.max',3)\
        .getOrCreate()

spark

In [ ]:
# get spark context -> entry point used to work with RDD
sc = spark.sparkContext
sc

In [ ]:
import os

df = spark.read.format('org.dianahep.sparkroot').load(file_list[0])

print(df.count())

In [ ]:
sqlContext = SQLContext(sc)

sqlContext.read.format('org.dianahep.sparkroot').option('tree', 'Events')\
.load('file:'+tofile)

In [ ]:
!$SPARK_HOME/bin/pyspark --master spark://ip-10.67.22.59:7077 --packages org.apache.hadoop:hadoop-aws:2.7.0

In [ ]:
#Read the ROOT file into a Spark DataFrame
df = spark.read\
    .format('org.dianahep.sparkroot')\
    .load('s3://' + buck + tofile)
# ... and print the number of events
# print df.count()

#### Verifying spark

In [13]:
# python dataset
data = [1,2,3,4,5,6,7,8]

# parallelize
sc.parallelize(data).count()

8

In [ ]:
import random

def inside(p):     
    x, y = random.random(), random.random()
    return x*x + y*y < 1

num_samples = 1000000
count = sc.parallelize(range(0, num_samples)).filter(inside).count()
pi = 4.0 * count / num_samples
print(pi)

#### Stop cluster

In [14]:
sc.stop()